# Selenium 구글 이미지 스크래핑

## Selenium 및 웹 드라이버 설치

In [11]:
!pip install Selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dec-shm-usage')

wd = webdriver.Chrome('chromedriver', options = chrome_options)

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:4 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Ign:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Fetched 252 kB in 2s (152 kB/s)
Reading package lists... Done
Reading package lists... Done
Building de

## 필요 라이브러리 import

In [12]:
import os
import time
import socket

from urllib.request import urlretrieve
from urllib.error import HTTPError, URLError
from selenium.common.exceptions import ElementClickInterceptedException, NoSuchElementException, ElementNotInteractableException
from PIL import Image


#scroll_down(): 스크롤을 내리는 함수

In [21]:
def scroll_down():
    scroll_count = 0
    print('[scroll_down(): 스크롤 다운 시작]')

    last_height = wd.execute_script('return document.body.scrollHeihgt')
    after_click = False

    while True:
        print(f'[스크롤 다운: {scroll_count}]')
        wd.execute_script('window.scrollTo(0,document.body.scrollHeight);')
        scroll_count += 1
        time.sleep(1)

        new_height = wd.execute_script('return document.body.scrollHeight')

        if last_height == new_height:
            if after_click is True:
                break

            else:
                try:
                    more_button = wd.find_element_by_xpath('//*[@id="islmp"]/div/div/div/div/div[5]/input')
                    if more_button.is_displayed():
                        more_button.click()
                        after_click = True
                except NoSuchElementException as e:
                    print(e)
                    break
        
        last_height = new_height





## click_and_save(): 썸네일 이미지 선택 후 원본 이미지 저장

In [60]:
def click_and_save(dir_name, index, img,img_list_length):
    global scraped_count

    try:
        img.click()
        wd.implicitly_wait(3)
        src = wd.find_element_by_xpath('//*[@id="Sva75c"]/div/div/div[3]/div[2]/c-wiz/div[1]/div[1]/div/div[2]/a/img').get_attribute('src')

        if src.split('.')[-1] == 'png':
            urlretrieve(src, dir_name+ '/' +str(scraped_count + 1) + '.png')
            print(f'{index+1}/{img_list_length} PNG 이미지 저장')

        else:
            urlretrieve(src, dir_name+ '/' +str(scraped_count + 1) + '.jpg')
            print(f'{index+1}/{img_list_length} JPG 이미지 저장')

        scraped_count += 1
    
    except HTTPError as e:
        print(e)
        pass

    except  ElementClickInterceptedException as e:
        print(e)
        pass


## scraping(): 구글 이미지 스크래핑 함수

In [59]:
def  scraping(dir_name, query):
    global scraped_count

    url = f'https://www.google.com/search?q={query}&tbm=isch&hl=ko&hl=ko&tbs=isz%3Al&sa=X&ved=0CAEQpwVqFwoTCIDcvM-ZxOsCFQAAAAAdAAAAABAC&biw=750&bih=727'
    print(url)
    wd.get(url)
    wd.maximize_window()

    scroll_down()

    div = wd.find_element_by_xpath('//*[@id="islrg"]/div[1]')
    print(div)
    # img_list = div.find_elements_by_xpath('//*[@id="islrg"]/div[1]/div[1]/a[1]/div/img')
    # img_list = div.find_elements_by_css_selector('.rg_i Q4LuWd')
    img_list = div.find_elements_by_css_selector('div.bRMDJf.islir > img')
    
    print(img_list)
    
    for index, img in enumerate(img_list):
        try:
            click_and_save(dir_name, index, img, len(img_list))
        except  ElementClickInterceptedException as e:
            print(e)
            wd.execute_script('window.scrollTo(0, window.scrollY+ 100')
            time.sleep(1)
            click_and_save(dir_name, index, img, len(img_list))
        except ConnectionResetError as e:
            print(e)
            pass
        except URLError as e:
            print(e)
            pass
        except socket.timeout as e:
            print(e)
            pass
        except socket.gaierror as e:
            print(e)
            pass
        except ElementNotInteractableException as e:
            print(e)
            break

    try:
        print('[스크래핑 종료 (성공률: %.2f%%)]' % (scraped_count / len(img_list) * 100.0))
    except ZeroDivisionError as e:
        print(e)

    wd.quit()

## filter_and_remove(): 일정 해상도 이하이거나 손상된 이미지 제고

In [29]:
def filter_and_remove(dir_name,query, filter_size):
    filtered_count = 0
    
    for index, fild_name in enumerate(os.listdir(dir_name)):
        try:
            file_path = os.path.join(dir_name, fild_name)
            img = Image.open(file_path)

            if img.width < filter_size and img.height < filter_size:
                img.close()
                os.remove(file_path)
                print(f'{index} 이미지 제거')
                filtered_count += 1
        except OSError as e:
            print(e)
            os.remove(file_path)
            filtered_count += 1
        
    print(f'[이미지 제거 개수: {filtered_count}/{scraped_count}]')

        

##스크래핑 실행

In [63]:
socket.setdefaulttimeout(30)

wd = webdriver.Chrome('chromedriver', options = chrome_options)
scraped_count = 0

path = './'
query = input('검색어: ')

dir_name = path+query
os.makedirs(dir_name)
print(f'[{dir_name} 디렉토리 생성]')

scraping(dir_name, query)           
filter_and_remove(dir_name, query, 400)

검색어: forest
[./forest 디렉토리 생성]
https://www.google.com/search?q=forest&tbm=isch&hl=ko&hl=ko&tbs=isz%3Al&sa=X&ved=0CAEQpwVqFwoTCIDcvM-ZxOsCFQAAAAAdAAAAABAC&biw=750&bih=727
[scroll_down(): 스크롤 다운 시작]
[스크롤 다운: 0]
[스크롤 다운: 1]
[스크롤 다운: 2]
[스크롤 다운: 3]
[스크롤 다운: 4]
[스크롤 다운: 5]
[스크롤 다운: 6]
[스크롤 다운: 7]
[스크롤 다운: 8]
<selenium.webdriver.remote.webelement.WebElement (session="6af34b1510d7dd9d760379ef39d38087", element="15cfda2a-a6ea-4e75-a282-44d7bc950cb6")>
[<selenium.webdriver.remote.webelement.WebElement (session="6af34b1510d7dd9d760379ef39d38087", element="891145d6-a57b-4410-a615-5a753d8dd234")>, <selenium.webdriver.remote.webelement.WebElement (session="6af34b1510d7dd9d760379ef39d38087", element="d246688c-43fc-43b1-bd10-a0dd20eacb65")>, <selenium.webdriver.remote.webelement.WebElement (session="6af34b1510d7dd9d760379ef39d38087", element="2b15f6e4-1e70-4c3a-8813-1de61f950270")>, <selenium.webdriver.remote.webelement.WebElement (session="6af34b1510d7dd9d760379ef39d38087", element="017135b5-e902-41

In [64]:
from google.colab import files
files.download('./sea/10.jpg')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [61]:
!ls

sample_data  sea


In [65]:
!ls forest  # 높고 조흔 품질의.. 안좋은것들은 제고하고..

104.jpg  186.jpg  261.jpg  321.jpg  40.jpg   479.jpg  540.jpg  59.jpg	72.jpg
106.jpg  187.jpg  264.jpg  322.jpg  414.jpg  480.jpg  543.jpg  5.jpg	76.jpg
110.jpg  197.jpg  265.jpg  33.jpg   41.jpg   483.jpg  544.jpg  605.jpg	79.jpg
115.jpg  200.jpg  26.jpg   342.jpg  420.jpg  485.jpg  545.jpg  606.jpg	91.jpg
116.jpg  215.jpg  271.jpg  351.jpg  422.jpg  486.jpg  546.jpg  609.jpg	92.jpg
117.jpg  21.jpg   272.jpg  353.jpg  426.jpg  491.jpg  549.jpg  60.jpg	93.jpg
118.jpg  226.jpg  279.jpg  362.jpg  431.jpg  492.jpg  552.jpg  610.jpg	94.jpg
124.jpg  227.jpg  285.jpg  363.jpg  432.jpg  493.jpg  553.jpg  619.jpg	97.jpg
129.jpg  233.jpg  287.jpg  36.jpg   434.jpg  494.jpg  558.jpg  61.jpg	98.jpg
12.jpg	 237.jpg  288.jpg  375.jpg  436.jpg  4.jpg    562.jpg  632.jpg	99.jpg
151.jpg  238.jpg  292.jpg  380.jpg  450.jpg  500.jpg  56.jpg   634.jpg
152.jpg  240.jpg  296.jpg  383.jpg  455.jpg  506.jpg  585.jpg  649.jpg
155.jpg  242.jpg  297.jpg  384.jpg  459.jpg  511.jpg  587.jpg  65.jpg
163.jpg  250.j

In [67]:
from google.colab import files
files.download('./forest/12.jpg')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [62]:
!ls sea

10.jpg	16.jpg	21.jpg	27.jpg	32.jpg	38.jpg	43.jpg	49.jpg	7.jpg
11.jpg	17.jpg	22.jpg	28.jpg	33.jpg	39.jpg	44.jpg	4.jpg	8.jpg
12.jpg	18.jpg	23.jpg	29.jpg	34.jpg	3.jpg	45.jpg	50.jpg	9.jpg
13.jpg	19.jpg	24.jpg	2.jpg	35.jpg	40.jpg	46.jpg	51.jpg
14.jpg	1.jpg	25.jpg	30.jpg	36.jpg	41.jpg	47.jpg	5.jpg
15.jpg	20.jpg	26.jpg	31.jpg	37.jpg	42.jpg	48.jpg	6.jpg


In [ ]:
from google.colab import files
files.download('./sea/10.jpg')

In [56]:
!rmdir sea

rmdir: failed to remove 'sea': Directory not empty


In [57]:
!rm -rf sea

## `scroll_down()`: 스크롤을 내리는 함수

## `click_and_save()`: 썸네일 이미지 선택 후 원본 이미지 저장

## `scraping()`: 구글 이미지 스크래핑 시작

## `filter_and_remove()`: 일정 해상도 이하이거나 손상된 이미지 제거

## 이미지 확인